In [1]:
from elasticsearch import Elasticsearch
from elasticsearch.helpers import bulk, streaming_bulk
import json
import math
import numbers

In [ ]:
host_url = ['http://127.0.0.1:9200']

In [ ]:
es_conn = Elasticsearch(host_url)

In [ ]:
def sanitize(value):
    if isinstance(value, numbers.Number) and math.isnan(value):
        return None
    else:
        return value

def parse_metadata(metadata_df):
    for index, row in metadata_df.iterrows():
        yield json.dumps({'index' : 'metadata',
        'doc_type' : 'record',
        'doc' : {
        'schema' : sanitize(row[0]),
        'uuid' : sanitize(row[1]), 
        'id' : sanitize(row[2]), 
        'title' : sanitize(row[3]), 
        'datasetcreationdate' : sanitize(row[4]), 
        'abstract' : sanitize(row[5]),
        'keyword' : sanitize(row[6]),
        'geoBox' : sanitize(row[7]),
        'temporalExtent' : sanitize(row[8]),
        'image' : sanitize(row[9]),
        'responsibleParty' : sanitize(row[10]),
        'link' : sanitize(row[11]),
        'metadatacreationdate' : sanitize(row[12]),
        'LegalConstraints' : sanitize(row[13])
              },}, 
        allow_nan = False)

In [ ]:
df = pd.read_csv('clean_metadata.csv')

In [ ]:
bulk(es_conn, parse_metadata(df), index = 'metadata', doc_type = 'record')

In [2]:
print (es_conn.count(index='metadata')['count'], 'documents in index')

NameError: name 'es_conn' is not defined